Notebook copied from: https://www.kaggle.com/mlanhenke/tps-11-nn-baseline-keras

In [ ]:
import os
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datatable as dt
from tensorflow import keras
from tensorflow.random import set_seed
from warnings import filterwarnings
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, InputLayer, Dropout
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.models import load_model
from sklearn.model_selection import StratifiedKFold

filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [ ]:
### set seeds
my_seed = 1

np.random.seed(my_seed)
set_seed(my_seed)

In [ ]:
from sklearn.preprocessing import StandardScaler

df_train = dt.fread('../input/tabular-playground-series-nov-2021/train.csv').to_pandas()
df_test = dt.fread('../input/tabular-playground-series-nov-2021/test.csv').to_pandas()
sample_submission = pd.read_csv('../input/tabular-playground-series-nov-2021/sample_submission.csv')

X = df_train.drop(columns=['id','target']).copy()
y = df_train['target'].copy()
X_test = df_test.drop(columns='id').copy()

scaler = StandardScaler()
X = pd.DataFrame(columns=X.columns, data=scaler.fit_transform(X))
X_test = pd.DataFrame(columns=X_test.columns, data=scaler.transform(X_test))

### LR Callback
Learning rate callback with warmup, dual piecewise learning rate, followed by learning rate decay

In [ ]:
def lr_nn_callback():
    lr_start    = 5e-5
    lr_max1     = 0.002
    lr_max2     = 0.001
    lr_min      = 0
    lr_ramp_ep  = 5
    lr_sus_ep1  = 15
    lr_sus_ep2  = 15
    lr_decay    = 0.85
    lr_decay_ep = 1
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max1 - lr_start) / lr_ramp_ep * epoch + lr_start   
        elif epoch < lr_ramp_ep + lr_sus_ep1:
            lr = lr_max1        
        elif epoch < lr_ramp_ep + lr_sus_ep1 + lr_sus_ep2:
            lr = lr_max2 
        else:
            lr = (lr_max2 - lr_min) * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep1 - lr_sus_ep2)//lr_decay_ep) + lr_min    
        return lr

    lr_callback = LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=15, 
    verbose=0,
    mode='min', 
    baseline=None, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2,
    patience=5,
    mode='min'
)

In [ ]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=my_seed)

scores = {fold:None for fold in range(cv.n_splits)}
predictions = []

print('Training Started.')
for fold, (idx_train, idx_valid) in enumerate(cv.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]

    model = load_model('../input/nov-baseline-nn/base_model')
    
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC']
    )
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        batch_size=4000,
        epochs=200,
        verbose=2,
        callbacks=[
            early_stopping,
            reduce_lr,
        ]
    )
    
    print(f"Fold {fold+1} || Best Validation AUC: {np.max(history.history['val_auc']):05f}")
    
    prediction = model.predict(X_test).reshape(1,-1)[0]
    predictions.append(prediction)

print('Training Finished')

Best
AUC:  0.759198

In [ ]:
sample_submission['target'] = np.mean(np.column_stack(predictions), axis=1)
sample_submission.to_csv('./submission.csv', index=False)